# Determine where to start for a coffee business which is close to a library or bookstore? 

Based on the previous lab assignment, I further explore the areas in Toronto. 

Load dataframe with geographical coordinates 

In [259]:
import pandas as pd
tdata = pd.read_csv('torontogeodata.csv')
tdata.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Create a map of Toronto

In [260]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

address = 'Toronto',
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

import folium # map rendering library

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(tdata['Latitude'], tdata['Longitude'], tdata['Borough'], tdata['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
folium.features.Choropleth(geo_data='neighbourhoods_toronto.geojson', 
                           line_color='blue',line_weight=1,
                           fill_color='#bbb').add_to(map_toronto)
map_toronto

Define Foursquare Credentials and Version


In [261]:
CLIENT_ID = 'VGCH045S115QKQQXJ3RCGK3KLAWHI1Q0JOSQW1AYJAWRH4RJ' # your Foursquare ID
CLIENT_SECRET = 'EEHK3KE1WS1NWEPMYIOMNCYWZIYJB41ZW3XOFPPO25NHDQVB' # your Foursquare Secret
VERSION = '20190417' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VGCH045S115QKQQXJ3RCGK3KLAWHI1Q0JOSQW1AYJAWRH4RJ
CLIENT_SECRET:EEHK3KE1WS1NWEPMYIOMNCYWZIYJB41ZW3XOFPPO25NHDQVB


## Explore the neighborhoods in Toronto

Create a function to find the nearby venues of a neighborhood：
the top 50 venues around the first neighbourhood within a radius of 500 meters.

In [262]:
# !pip install requests

In [263]:
    import requests
    url = "https://github.com/timeline.json"
    r = requests.get(url)    
    print(r.json())
    print(type(r.json()))

{'message': 'Hello there, wayfaring stranger. If you’re reading this then you probably didn’t see our blog post a couple of years back announcing that this API would go away: http://git.io/17AROg Fear not, you should be able to get what you need from the shiny new Events API instead.', 'documentation_url': 'https://developer.github.com/v3/activity/events/#list-public-events'}
<class 'dict'>


In [264]:

radius = 500
LIMIT = 50

import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if results == '':
            results == 'NaN'
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue CategoryID']
    
    return(nearby_venues)

In [265]:
toronto_venues = getNearbyVenues(names= tdata['Neighbourhood'], latitudes= tdata['Latitude'], longitudes= tdata['Longitude'])

In [266]:
# check the size of dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1699, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue CategoryID
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,4bf58dd8d48988d16e941735
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,4bf58dd8d48988d116941735
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum,4bf58dd8d48988d190941735
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place,4bf58dd8d48988d1ca941735
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,4bf58dd8d48988d122951735


Check the number of unique categories can be curated from all the returned venues

In [267]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 255 uniques categories.


### Step 1. Determine similar business by specifying category ID: 
I'm just looking for the coffee shops including: 
Cafeteria: 4bf58dd8d48988d128941735 ||
Café: 4bf58dd8d48988d16d941735  ||
Coffee Shop: 4bf58dd8d48988d1e0931735

In [268]:
toronto_cafeshops = toronto_venues.loc[ (toronto_venues['Venue CategoryID']=='4bf58dd8d48988d128941735') |
                                   (toronto_venues['Venue CategoryID']=='4bf58dd8d48988d16d941735')|
                                   (toronto_venues['Venue CategoryID']=='4bf58dd8d48988d1e0931735') ]
toronto_cafeshops.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue CategoryID
11,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop,4bf58dd8d48988d1e0931735
12,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop,4bf58dd8d48988d1e0931735
24,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop,4bf58dd8d48988d1e0931735
41,"Birch Cliff,Cliffside West",43.692657,-79.264848,The Birchcliff,43.691666,-79.264532,Café,4bf58dd8d48988d16d941735
78,L'Amoreaux West,43.799525,-79.318389,Tim Hortons,43.798281,-79.318317,Coffee Shop,4bf58dd8d48988d1e0931735


In [269]:
# check the total number of coffee shops 
toronto_cafeshops.shape

(212, 8)

In [270]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Neighborhood Latitude'], df['Neighborhood Longitude'], df['Neighborhood'], 
                                                df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

Mark the coffee shops 

In [271]:
map_toronto_coffee = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(toronto_cafeshops, 'red', map_toronto_coffee)
map_toronto_coffee

### Step 2. Determine library or bookstore locations by specifying category ID: 

I'm just looking for the bookstores or library including: 
library: 4bf58dd8d48988d12f941735 ||
College Library: 4bf58dd8d48988d1a7941735  ||
Bookstore: 4bf58dd8d48988d114951735 ||
Used Bookstore: 52f2ab2ebcbc57f1066b8b30 ||
College Bookstore: 4bf58dd8d48988d1b1941735

In [272]:
toronto_lib = toronto_venues.loc[ (toronto_venues['Venue CategoryID']=='4bf58dd8d48988d12f941735') |
                                   (toronto_venues['Venue CategoryID']=='4bf58dd8d48988d1a7941735')|
                                   (toronto_venues['Venue CategoryID']=='4bf58dd8d48988d114951735') |
                                  (toronto_venues['Venue CategoryID']=='52f2ab2ebcbc57f1066b8b30') |
                                   (toronto_venues['Venue CategoryID']=='4bf58dd8d48988d1b1941735')]
toronto_lib.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue CategoryID
361,"The Danforth West,Riverdale",43.679557,-79.352188,Re: Reading,43.678507,-79.347678,Bookstore,4bf58dd8d48988d114951735
384,"The Danforth West,Riverdale",43.679557,-79.352188,Book City,43.677413,-79.352734,Bookstore,4bf58dd8d48988d114951735
411,Studio District,43.659526,-79.340923,Queen Books,43.660651,-79.342267,Bookstore,4bf58dd8d48988d114951735
575,"Cabbagetown,St. James Town",43.667967,-79.367675,International News,43.665516,-79.368490,Bookstore,4bf58dd8d48988d114951735
581,Church and Wellesley,43.665860,-79.383160,Glad Day Bookshop,43.665271,-79.380785,Bookstore,4bf58dd8d48988d114951735


In [273]:
# check the total number of coffee shops 
toronto_lib.shape

(13, 8)

In [274]:
map_toronto_lib = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(toronto_lib, 'red', map_toronto_lib)
map_toronto_lib

### Step 3. Determine the neighborhood with the highest number of bookstores and libraries

In [275]:
toronto_lib_groupby = toronto_lib.groupby(toronto_lib['Neighborhood'])
toronto_lib_groupby.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue CategoryID
361,"The Danforth West,Riverdale",43.679557,-79.352188,Re: Reading,43.678507,-79.347678,Bookstore,4bf58dd8d48988d114951735
384,"The Danforth West,Riverdale",43.679557,-79.352188,Book City,43.677413,-79.352734,Bookstore,4bf58dd8d48988d114951735
411,Studio District,43.659526,-79.340923,Queen Books,43.660651,-79.342267,Bookstore,4bf58dd8d48988d114951735
575,"Cabbagetown,St. James Town",43.667967,-79.367675,International News,43.665516,-79.368490,Bookstore,4bf58dd8d48988d114951735
581,Church and Wellesley,43.665860,-79.383160,Glad Day Bookshop,43.665271,-79.380785,Bookstore,4bf58dd8d48988d114951735
1006,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,Indigospirit,43.648350,-79.380347,Bookstore,4bf58dd8d48988d114951735
1066,"Commerce Court,Victoria Hotel",43.648198,-79.379817,Indigospirit,43.648350,-79.380347,Bookstore,4bf58dd8d48988d114951735
1133,"Harbord,University of Toronto",43.662696,-79.400049,Bakka Phoenix Books,43.662959,-79.402601,Bookstore,4bf58dd8d48988d114951735
1142,"Harbord,University of Toronto",43.662696,-79.400049,Coach House Printing,43.666320,-79.400277,Bookstore,4bf58dd8d48988d114951735
1315,"First Canadian Place,Underground city",43.648429,-79.382280,Indigospirit,43.648350,-79.380347,Bookstore,4bf58dd8d48988d114951735


In [276]:
toronto_lib_g = toronto_lib_groupby['Venue'].count()
toronto_lib_g = toronto_lib_g.to_frame(name = None)
toronto_lib_g.reset_index(inplace = True)
toronto_lib_g

,Neighborhood,Venue
0,"Cabbagetown,St. James Town",1
1,Church and Wellesley,1
2,"Commerce Court,Victoria Hotel",1
3,"Design Exchange,Toronto Dominion Centre",1
4,"First Canadian Place,Underground city",1
5,"Harbord,University of Toronto",2
6,"High Park,The Junction South",1
7,"Parkdale,Roncesvalles",1
8,"Runnymede,Swansea",1
9,Studio District,1


In [277]:
toronto_lib_g = toronto_lib_g.rename(columns={'Neighborhood':'Neighborhood','Venue':'Count' })
toronto_lib_g

,Neighborhood,Count
0,"Cabbagetown,St. James Town",1
1,Church and Wellesley,1
2,"Commerce Court,Victoria Hotel",1
3,"Design Exchange,Toronto Dominion Centre",1
4,"First Canadian Place,Underground city",1
5,"Harbord,University of Toronto",2
6,"High Park,The Junction South",1
7,"Parkdale,Roncesvalles",1
8,"Runnymede,Swansea",1
9,Studio District,1


In [278]:
toronto_lib_g.sort_values(by=['Count'], ascending = False)

,Neighborhood,Count
5,"Harbord,University of Toronto",2
10,"The Danforth West,Riverdale",2
0,"Cabbagetown,St. James Town",1
1,Church and Wellesley,1
2,"Commerce Court,Victoria Hotel",1
3,"Design Exchange,Toronto Dominion Centre",1
4,"First Canadian Place,Underground city",1
6,"High Park,The Junction South",1
7,"Parkdale,Roncesvalles",1
8,"Runnymede,Swansea",1


### The above table indicates that one should select either 'Harbord,University of Toronto' or 'The Danforth West,Riverdale'

### Step 4. Determine the number of coffee shops in the above two specified neighborhood

In [279]:
toronto_cafeshops_groupby = toronto_cafeshops.groupby(toronto_cafeshops['Neighborhood'])
toronto_cafeshops_groupby.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue CategoryID
11,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop,4bf58dd8d48988d1e0931735
12,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop,4bf58dd8d48988d1e0931735
24,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop,4bf58dd8d48988d1e0931735
41,"Birch Cliff,Cliffside West",43.692657,-79.264848,The Birchcliff,43.691666,-79.264532,Café,4bf58dd8d48988d16d941735
78,L'Amoreaux West,43.799525,-79.318389,Tim Hortons,43.798281,-79.318317,Coffee Shop,4bf58dd8d48988d1e0931735
107,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,Aroma Espresso Bar,43.777700,-79.344652,Coffee Shop,4bf58dd8d48988d1e0931735
119,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,Starbucks,43.778287,-79.343331,Coffee Shop,4bf58dd8d48988d1e0931735
122,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,Tim Hortons,43.774965,-79.346285,Coffee Shop,4bf58dd8d48988d1e0931735
130,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,Tim Hortons,43.775249,-79.347740,Coffee Shop,4bf58dd8d48988d1e0931735
148,Bayview Village,43.786947,-79.385975,Maxim's Cafe and Patisserie,43.787863,-79.380751,Café,4bf58dd8d48988d16d941735


In [280]:
toronto_cafeshop_g = toronto_cafeshops_groupby['Venue'].count()
toronto_cafeshop_g = toronto_cafeshop_g.to_frame(name = None)
toronto_cafeshop_g.reset_index(inplace = True)
toronto_cafeshop_g.head()

,Neighborhood,Venue
0,"Adelaide,King,Richmond",6
1,"Alderwood,Long Branch",1
2,"Bathurst Manor,Downsview North,Wilson Heights",2
3,Bayview Village,1
4,"Bedford Park,Lawrence Manor East",3


In [281]:
toronto_cafeshop_g = toronto_cafeshop_g.rename(columns={'Neighborhood':'Neighborhood','Venue':'Count' })
toronto_cafeshop_g.head()

,Neighborhood,Count
0,"Adelaide,King,Richmond",6
1,"Alderwood,Long Branch",1
2,"Bathurst Manor,Downsview North,Wilson Heights",2
3,Bayview Village,1
4,"Bedford Park,Lawrence Manor East",3


In [282]:
coffeestats = toronto_cafeshop_g.sort_values(by=['Count'], ascending = False)
coffeestats.head()

,Neighborhood,Count
18,"Design Exchange,Toronto Dominion Centre",13
15,"Commerce Court,Victoria Hotel",12
38,Queen's Park,10
28,"Harbourfront,Regent Park",10
11,Central Bay Street,10


In [283]:
coffeestats.loc[coffeestats['Neighborhood']=='Harbord,University of Toronto']

,Neighborhood,Count
26,"Harbord,University of Toronto",6


In [284]:
coffeestats.loc[coffeestats['Neighborhood']=='The Danforth West,Riverdale']

,Neighborhood,Count
47,"The Danforth West,Riverdale",5


# Conclusion: The Danforth West,Riverdale  (East Toronto, post code: M4K) is the optimal neighborhood to open a coffee shop

create a map of M4K area

In [285]:
lat_m4k = 43.6668
lon_m4k = -79.3501
map_m4k = folium.Map(location=[lat_m4k, lon_m4k], zoom_start=12)
folium.vector_layers.Circle(location=[lat_m4k, lon_m4k], radius=100, 
                            popup='best location', tooltip= 'best location', 
                            ).add_to(map_m4k)
# folium.features.Choropleth(geo_data='neighbourhoods_toronto.geojson', 
#                            line_color='blue',line_weight=1,
#                            fill_color='YlOrRd').add_to(map_m4k)
addToMap(toronto_cafeshops, 'red', map_m4k)
addToMap(toronto_lib, 'green', map_m4k)
map_m4k